# How to build an advanced Chatbot with session memory using LangChain
* Advanced Chatbot LLM App.
    * Will be able to have a conversation.
    * Will remember previous interactions: will have memory.
    * Will be able to have different memories for different user sessions.
    * Will be able to remember a limited number of messages: limited memory.


## Concepts included
* Chat Model vs. LLM Model:
    *  Chat Model is based around messages.
    *  LLM Model is based around raw text.
* Chat History: allows Chat Model to remember previous interactions.


#### What is BaseChatMessageHistory and what it does?
BaseChatMessageHistory is what is called an **abstract base class** in Python. [See more info about this here](https://api.python.langchain.com/en/latest/chat_history/langchain_core.chat_history.BaseChatMessageHistory.html). This means it serves as a template or a foundational **blueprint for other classes**. It outlines a set of methods and structures that any class inheriting from it must implement or adhere to, but it cannot be used to create objects directly.

Here's a simpler breakdown of what it means for `BaseChatMessageHistory` to be an abstract base class:

1. **Blueprint for Other Classes:** It provides a predefined structure that other classes can follow. Think of it like an outline or a checklist for building something; it specifies what needs to be included, but it isn’t the final product.

2. **Cannot Create Instances:** You cannot create an instance of an abstract base class. Trying to create an object directly from `BaseChatMessageHistory` would result in an error because it's meant to be a guide, not something to use directly.

3. **Requires Implementation:** Any class that inherits from this abstract base class needs to implement specific methods outlined in `BaseChatMessageHistory`, such as methods for adding messages, retrieving messages, and clearing messages. The class sets the rules, and the subclasses need to follow these rules by providing the actual operational details.

4. **Purpose in Design:** Using an abstract base class helps ensure consistency and correctness in the implementation of classes that extend it. It's a way to enforce certain functionalities in any subclass, making sure that they all behave as expected without rewriting the same code multiple times.

Overall, the concept of an abstract base class is about setting standards and rules, while leaving the specific details of execution to be defined by subclasses that inherit from it.


#### Let's explain the previous code in simple terms
The previous code manages the chatbot's memory of conversations based on session identifiers. Here’s a breakdown of what the different components do:

1. **chatbotMemory**:
    - `chatbotMemory = {}`: This initializes an empty dictionary where session IDs and their respective chat histories will be stored.

2. **get_session_history Function**:
    - This function, `get_session_history`, takes a `session_id` as an argument and returns the chat history associated with that session.
    - If a chat history for the given `session_id` does not exist in `chatbotMemory`, a new instance of `ChatMessageHistory` is created and assigned to that `session_id` in the dictionary.
    - The function ensures that each session has its own unique chat history, stored and retrieved using the session ID.

3. **chatbot_with_message_history**:
    - `chatbot_with_message_history = RunnableWithMessageHistory(chatbot, get_session_history)`: This line creates an instance of `RunnableWithMessageHistory` using two arguments: `chatbot` and `get_session_history`.
    - The `chatbot` is passed along with the `get_session_history` function. This setup integrates the chatbot with the functionality to handle session-specific chat histories, allowing the chatbot to maintain continuity and context in conversations across different sessions.
    - **Learn more about RunnableWithMessageHistory** [here](https://python.langchain.com/v0.1/docs/expression_language/how_to/message_history/).

Overall, the code organizes and manages a chatbot's memory, enabling it to handle different sessions with users effectively by remembering previous messages within each session.



In [1]:
#Avoid legacy Langchain depreciation warning
import warnings
from langchain._api import LangChainDeprecationWarning

warnings.simplefilter("ignore", category=LangChainDeprecationWarning)

In [2]:
#load my env variable
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
groq_api_key = os.environ["GROQ_API_KEY"]

In [3]:
#load our chat completion model
from langchain_groq import ChatGroq

chatbot = ChatGroq(model="mixtral-8x7b-32768")

In [4]:
#Let's check with simple prompt - human prompt
from langchain_core.messages import HumanMessage

messagesToTheChatbot = [
    HumanMessage(content="My favourite place is Switzerland"),
]

In [5]:
#Let's pass to the LLM with the help of invoke function
#Response in the form of AIMessage
chatbot.invoke(messagesToTheChatbot)

AIMessage(content="Switzerland is a beautiful country located in the heart of Europe, known for its stunning landscapes, clean cities, and rich culture. Here are a few reasons why Switzerland might be your favorite place:\n\n1. Natural Beauty: Switzerland is home to some of the most breathtaking landscapes in the world, from the majestic Swiss Alps to the crystal-clear lakes and rivers. Whether you enjoy hiking, skiing, or simply taking in the views, Switzerland has something for everyone.\n2. Cultural Diversity: Switzerland is a multilingual and multicultural country, with four official languages (German, French, Italian, and Romansh) and a diverse population. This means that you can experience different cultures, traditions, and cuisines all within one country.\n3. Safety and Cleanliness: Switzerland is one of the safest and cleanest countries in the world, with a high standard of living and a strong emphasis on environmental sustainability. This makes it an ideal destination for tra

In [6]:
#Let's check our chatbot remembers our favourite place
chatbot.invoke([
    HumanMessage(content="Do you know my favourite place?")
])

AIMessage(content="I'm afraid I don't have enough information to know your favorite place. You can tell me more about it, if you'd like. I can assist you with information about places, if that's what you're looking for.", response_metadata={'token_usage': {'completion_tokens': 52, 'prompt_tokens': 15, 'total_tokens': 67, 'completion_time': 0.081630024, 'prompt_time': 0.002572343, 'queue_time': 0.028139852, 'total_time': 0.084202367}, 'model_name': 'mixtral-8x7b-32768', 'system_fingerprint': 'fp_c5f20b5bb1', 'finish_reason': 'stop', 'logprobs': None}, id='run-3188bd78-d0c9-4494-9672-bf28258084d9-0', usage_metadata={'input_tokens': 15, 'output_tokens': 52, 'total_tokens': 67})

In [8]:
#Let's add memory to our Chatbot

from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

chatbotMemory = {}

# input: session_id, output: chatbotMemory[session_id]
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in chatbotMemory:
        chatbotMemory[session_id] = ChatMessageHistory()
    return chatbotMemory[session_id]


chatbot_with_message_history = RunnableWithMessageHistory(
    chatbot, 
    get_session_history
)

In [10]:
#let's create the session - create different sessions for different users
#Session - person A cannot access the person B history
session1 = {"configurable": {"session_id": "001"}}

In [11]:
responseFromChatbot = chatbot_with_message_history.invoke(
    [HumanMessage(content="My favorite place is Switzerland.")],
    config=session1,
)

responseFromChatbot.content

"That's great! Switzerland is known for its stunning landscapes, including the Alps, picturesque towns, and delicious chocolate. Do you have a favorite city or region in Switzerland that you've visited or would like to visit?"

In [12]:
responseFromChatbot = chatbot_with_message_history.invoke(
    [HumanMessage(content="Which is my favorite place?")],
    config=session1,
)

responseFromChatbot.content

'My apologies, I had assumed based on your previous message that Switzerland is your favorite place. Is that incorrect, and you would like to share a different favorite place instead?'

In [13]:
#Let's change the session id
session2 = {"configurable": {"session_id": "002"}}

In [14]:
responseFromChatbot = chatbot_with_message_history.invoke(
    [HumanMessage(content="Which is my favorite place?")],
    config=session2,
)

responseFromChatbot.content

"I don't have personal experiences, emotions, or favorites. However, I can help you find information about your favorite place if you tell me what it is."

In [15]:
#Let's go back to session 1 and see if the memory is still there or not
session1 = {"configurable": {"session_id": "001"}}

In [16]:
responseFromChatbot = chatbot_with_message_history.invoke(
    [HumanMessage(content="Which is my favorite place?")],
    config=session1,
)

responseFromChatbot.content

'I apologize for any confusion. Based on the information you provided, you mentioned that Switzerland is your favorite place. If you have any other favorite places or would like to share more about your favorite things about Switzerland, I would be happy to hear more!'

In [17]:
### Our ChatBot has session memory now. Let's check if it remembers the conversation from session2.
session2 = {"configurable": {"session_id": "002"}}

In [18]:
responseFromChatbot = chatbot_with_message_history.invoke(
    [HumanMessage(content="My name is Amar.")],
    config=session2,
)

responseFromChatbot.content

"Hello Amar! It's nice to meet you. I'm here to help you with any questions or information you might need. If you'd like to know about your favorite place, could you please tell me where that is?"

In [19]:
responseFromChatbot = chatbot_with_message_history.invoke(
    [HumanMessage(content="What is my name?")],
    config=session2,
)

responseFromChatbot.content

'You mentioned earlier that your name is Amar. Is there something else you would like to know or discuss, Amar?'

In [20]:
responseFromChatbot = chatbot_with_message_history.invoke(
    [HumanMessage(content="Which is my favorite Place?")],
    config=session1,
)

responseFromChatbot.content

'I apologize if I am repeating myself, but based on the information you provided, you mentioned that Switzerland is your favorite place. If you have any other favorite places or would like to share more about your favorite things about Switzerland, I would be happy to hear more! If Switzerland is not your favorite place after all, please let me know and I will do my best to assist you with any questions you may have.'

## Our chatbot remembers each and every of our conversation

## The importance to manage the Conversation History
* The memory of a chatbot is included in the context window of the LLM so, if left unmanaged, can potentially overflow it.
* **We are now going to learn how to limit the size of the memory of a chatbot**.
* First, let's take a look at what is in the memory of our chatbot:

In [21]:
print(chatbotMemory)

{'001': InMemoryChatMessageHistory(messages=[HumanMessage(content='My favorite place is Switzerland.'), AIMessage(content="That's great! Switzerland is known for its stunning landscapes, including the Alps, picturesque towns, and delicious chocolate. Do you have a favorite city or region in Switzerland that you've visited or would like to visit?", response_metadata={'token_usage': {'completion_tokens': 49, 'prompt_tokens': 14, 'total_tokens': 63, 'completion_time': 0.077209845, 'prompt_time': 0.002056355, 'queue_time': 0.026210972, 'total_time': 0.0792662}, 'model_name': 'mixtral-8x7b-32768', 'system_fingerprint': 'fp_c5f20b5bb1', 'finish_reason': 'stop', 'logprobs': None}, id='run-d850989d-a550-48af-9150-fff93f5261b7-0', usage_metadata={'input_tokens': 14, 'output_tokens': 49, 'total_tokens': 63}), HumanMessage(content='Which is my favorite place?'), AIMessage(content='My apologies, I had assumed based on your previous message that Switzerland is your favorite place. Is that incorrect

In [22]:
#Now,let's define a function to limit the number of messages stored in memory and add it to our chain with .assign.

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnablePassthrough


def limited_memory_of_messages(messages, number_of_messages_to_keep=2):
    return messages[-number_of_messages_to_keep:]

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

limitedMemoryChain = (
    RunnablePassthrough.assign(messages=lambda x: limited_memory_of_messages(x["messages"]))
    | prompt 
    | chatbot
)

* The limited_memory_of_messages function allows you to trim the list of stored messages, keeping only a specified number of the latest ones. For example, if you have a long list of messages and you only want to keep the last two, this function will do that for you.
* The lambda function works in conjunction with the `limited_memory_of_messages` function. Here’s a simple breakdown:

    1. **Lambda Function**: The `lambda` keyword is used to create a small anonymous function in Python. The `lambda` function defined here takes one argument, `x`.

    2. **Function Argument**: The argument `x` is expected to be a dictionary that contains a key named `"messages"`. The value associated with this key is a list of messages.

    3. **Function Body**: The body of the `lambda` function calls the `limited_memory_of_messages` function. It passes the list of messages found in `x["messages"]` to this function.

    4. **Default Behavior of limited_memory_of_messages**: Since the `lambda` function does not specify the `number_of_messages_to_keep` parameter when it calls `limited_memory_of_messages`, the latter will default to keeping the last 2 messages from the list (as defined by the earlier function).

In essence, the `lambda` function is a shorthand way to apply the `limited_memory_of_messages` function to the message list contained within a dictionary. It automatically trims the list to the last two messages.

In [23]:
#Let's now create our new chatbot with limited message history

chatbot_with_limited_message_history = RunnableWithMessageHistory(
    limitedMemoryChain,
    get_session_history,
    input_messages_key="messages",
)

In [24]:
#Let's add 2 more messages to the session1 conversation:
responseFromChatbot = chatbot_with_message_history.invoke(
    [HumanMessage(content="My favorite car is Audi.")],
    config=session1,
)

responseFromChatbot.content

'Thank you for letting me know that your favorite car is Audi. Audi is a German luxury car manufacturer known for its high-performance vehicles and advanced technology. Do you have a particular model of Audi that is your favorite, or are you drawn to the brand as a whole? I would be happy to help you with any questions you may have about Audi or cars in general.'

In [25]:
responseFromChatbot = chatbot_with_message_history.invoke(
    [HumanMessage(content="My favorite sports is Cricket.")],
    config=session1,
)

responseFromChatbot.content

"That's great! Cricket is a popular bat-and-ball sport that originated in southeast England and is now played around the world. It is particularly popular in countries such as India, Pakistan, Australia, and England. Do you enjoy playing cricket or watching it as a spectator? If you play, what position do you prefer, and if you watch, do you have a favorite team or player? I would be happy to hear more about your interest in cricket."

In [27]:
## The chatbot memory has now 4 messages. Let's check the Chatbot with limited memory. 
# Remember, this chatbot only remembers the last 2 messages, so if we ask her about the first message she should not remember it.

responseFromChatbot = chatbot_with_limited_message_history.invoke(
    {
        "messages": [HumanMessage(content="what is my favorite place?")],
    },
    config=session1,
)

responseFromChatbot.content

"I'm an assistant designed to provide general information and assistance, and I don't have access to personal information about individuals, including their favorite places. If you'd like, I can help you find information about popular vacation spots, local attractions, or other places of interest."

In [28]:
#The chatbot with limited memory has behaved as we expected.

In [29]:
# Finally, let's compare the previous response with the one provided by the Chatbot with unlimited memory

responseFromChatbot = chatbot_with_message_history.invoke(
    [HumanMessage(content="which is my favorite place?")],
    config=session1,
)

responseFromChatbot.content

"I apologize for any confusion earlier. As a helpful assistant, I don't have access to personal information about individuals, so I don't know your favorite place. However, I can help you find information about different places if you'd like, or answer questions you have about travel, geography, or other topics."